# Analysing Trading212 Data

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
import IPython

In [2]:
file_path1 = 'input-data-file1'  
file_path2 = 'input-data-file2'
file_path3 = 'input-data-file3'

# Note: Trading212 only allows you to export data
# 1 year at a time.

df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)
df3 = pd.read_csv(file_path3)

# Combine the two DataFrames
combined_df = pd.concat([df1, df2, df3])

In [3]:
# Used because TSCO is a tractor company in the US

def change_ticker_name(df, original_ticker, new_ticker):
    df['Ticker'] = df['Ticker'].replace(original_ticker, new_ticker)
    return None

change_ticker_name(combined_df, 'TSCO', 'TSCO.L')

In [4]:
def get_current_price(ticker):
    try:
        stock = yf.Ticker(ticker)
        todays_data = stock.history(period='1d')
        price = todays_data['Close'][0]
        if ticker[-2:] == '.L':
            price /= 100
            return price
        else:
            return price
    except Exception as e:
        pass
        #print(f"Error fetching price for {ticker}: {e}")
    suffixes = [
        '.L',    # UK
        '.DE',   # Germany
        '.SW',   # Switzerland
        '.TO',   # Canada
        '.PA',   # France
        '.T',    # Japan
        '.AX',   # Australia
        '.HK'    # Hong Kong
    ]
    for suffix in suffixes:
        try:
            stock = yf.Ticker(ticker + suffix)
            todays_data = stock.history(period='1d')
            price = todays_data['Close'][0]
            # Divide the price by 100 if the suffix is ".L"
            if suffix == '.L':
                price /= 100
            return price
        except Exception as e:
            pass
            #print(f"Error fetching price for {ticker+suffix}: {e}")
    return None

## The full calculation

In [5]:
import pandas as pd
import yfinance as yf

# Load the CSV file
df = combined_df

# Create a dictionary of ticker symbols and their trades
trades = {}

for index, row in df.iterrows():
    ticker = row['Ticker']
    shares = row['No. of shares']
    total = row['Total (GBP)']

    if row['Action'] == 'Market buy':
        trades.setdefault(ticker, []).append(('buy', shares, total))
    elif row['Action'] == 'Market sell':
        trades.setdefault(ticker, []).append(('sell', shares, total))

# Calculate the final number of shares for each ticker
shares_owned = {}

for ticker, trades_list in trades.items():
    shares = 0

    for trade in trades_list:
        if trade[0] == 'buy':
            shares += trade[1]
        elif trade[0] == 'sell':
            shares -= trade[1]

    shares_owned[ticker] = shares

# Round the 'Shares owned' column to 6 decimal places and replace very small values with 0
df['Shares owned'] = df['Ticker'].apply(lambda x: round(shares_owned.get(x, 0), 7))
df['Shares owned'] = df['Shares owned'].apply(lambda x: 0 if abs(x) < 0.00000001 else x)

# Calculate the average price paid for each ticker
avg_price_paid = {}

for ticker, trades_list in trades.items():
    cost_basis = 0
    total_shares = 0

    for trade in trades_list:
        shares = trade[1]
        price_per_share = trade[2] / shares  # Compute the price per share for this trade
        cost_basis += (shares * price_per_share) 
        total_shares += shares

    if total_shares != 0:
        avg_price_paid[ticker] = cost_basis / total_shares
    else:
        avg_price_paid[ticker] = 0

In [6]:
# Calculate the current value and gain/loss for each ticker
current_value = {}
gain_loss = {}

for ticker, shares in shares_owned.items():
    if shares != 0:
        current_price = get_current_price(ticker)
        if current_price is None:
            current_value[ticker] = 'N/A'
            gain_loss[ticker] = 'N/A'
        else:
            current_value[ticker] = shares * current_price
            gain_loss[ticker] = current_value[ticker] - (shares * avg_price_paid[ticker])
    else:
        buy_trades = [trade for trade in trades[ticker] if trade[0] == 'buy']
        sell_trades = [trade for trade in trades[ticker] if trade[0] == 'sell']

        buy_cost = sum([trade[2] for trade in buy_trades])
        sell_proceeds = sum([trade[2] for trade in sell_trades])

        buy_shares = sum([trade[1] for trade in buy_trades])
        sell_shares = sum([trade[1] for trade in sell_trades])

        if buy_shares == 0:
            gain_loss[ticker] = -buy_cost
        elif sell_shares == 0:
            gain_loss[ticker] = 'N/A'
        else:
            buy_price = buy_cost / buy_shares
            sell_price = sell_proceeds / sell_shares
            gain_loss[ticker] = sell_proceeds - buy_cost

        current_value[ticker] = 'N/A' if current_price is None else shares * current_price
        
IPython.display.clear_output()

In [1]:
# Initialize the Total Invested and Total Gain variables
total_invested = 0
total_gain = 0

total_gain -= df['Withholding tax'].sum(skipna=True) + df['Stamp duty reserve tax (GBP)'].sum(skipna=True) + df['Currency conversion fee (GBP)'].sum(skipna=True)
# Iterate through the DataFrame
for index, row in df.iterrows():
    action = row['Action']
    total = row['Total (GBP)']
    withholding_tax = row['Withholding tax']
    stamp_duty_tax = row['Stamp duty reserve tax (GBP)']
    conversion_fee = row['Currency conversion fee (GBP)']

    # Update the Total Invested and Total Gain based on the action and specified conditions
    if action == 'Deposit':
        total_invested += total
    elif action == 'Withdrawal':
        total_invested -= total
    elif action == 'Dividend (Ordinary)' or action == 'Dividend (Bonus)':
        total_gain += total


# Print the Total Invested
print("")
print(f"Total Invested: {total_invested:.2f}")
print("")

# Print the Total Gain
print(f"Total Gain: {total_gain:.2f}")
print("")

# Print the results

# Print the results
for ticker in shares_owned.keys():
    if shares_owned[ticker] > 1e-13:
        print(f"Ticker: {ticker}")
        print(f"Shares owned: {df.loc[df['Ticker'] == ticker, 'Shares owned'].iloc[0]}")
        print(f"Average price paid: {avg_price_paid[ticker]:.2f}")
        if isinstance(current_value[ticker], str):
            print(f"Current value: {current_value[ticker]}")
        else:
            print(f"Current value: {current_value[ticker]:.2f}")
        if isinstance(gain_loss[ticker], str):
            print(f"Gain/loss: {gain_loss[ticker]}")
            print("")
        else:
            gain_loss_percentage = (gain_loss[ticker] / (shares_owned[ticker] * avg_price_paid[ticker])) * 100
            print(f"Gain/loss: {gain_loss[ticker]:.2f} ({gain_loss_percentage:.2f}%)")
            print("")

NameError: name 'df' is not defined